In [ ]:
!pip uninstall -y numpy jax jaxlib tensorflow
!pip install numpy==1.24.4 transformers==4.41.2 datasets pandas scikit-learn gradio --no-cache-dir --quiet
import os
os.kill(os.getpid(), 9)  



In [1]:
import numpy as np
print("✅ NumPy version:", np.__version__)


✅ NumPy version: 1.24.4


In [ ]:
!pip uninstall -y peft



In [2]:
from transformers import Trainer, TrainingArguments
print("✅ Hugging Face Transformers are now working!")


✅ Hugging Face Transformers are now working!


In [3]:
# ✅ 1. Load and preprocess dataset
import pandas as pd
from datasets import Dataset

train_df = pd.read_csv("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv", header=None)
test_df = pd.read_csv("https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv", header=None)

train_df.columns = ["label", "title", "description"]
test_df.columns = ["label", "title", "description"]

train_df["text"] = train_df["title"] + " " + train_df["description"]
test_df["text"] = test_df["title"] + " " + test_df["description"]

train_df["label"] -= 1
test_df["label"] -= 1

# Optional: rename for PyTorch clarity
train_df.rename(columns={"label": "labels"}, inplace=True)
test_df.rename(columns={"label": "labels"}, inplace=True)

train_dataset = Dataset.from_pandas(train_df[["text", "labels"]])
test_dataset = Dataset.from_pandas(test_df[["text", "labels"]])




In [4]:
# ✅ 2. Tokenize with padding/truncation
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
# ✅ 3. Load BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# ✅ 4. Metrics and training args
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments

def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(pred.label_ids, preds),
        "f1": f1_score(pred.label_ids, preds, average="weighted"),
    }

training_args = TrainingArguments(
    output_dir="./results",              # Saves model here
    eval_strategy="epoch",        # Evaluate every epoch
    save_strategy="epoch",              # Save every epoch
    per_device_train_batch_size=32,     # Increased batch size
    per_device_eval_batch_size=32,
    num_train_epochs=2,                 # Good for convergence
    logging_dir="./logs",               # Logging folder
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True,                          # Use FP16 on GPU for speed
    report_to="none"                    # Disable wandb/huggingface reporting unless you want it
)



In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # You already defined this before
)

# Start training with progress and metrics
print("🚀 Training started...")
train_output = trainer.train()
print("✅ Training complete.")

# Output final metrics
print("📊 Final Metrics:", train_output.metrics)


🚀 Training started...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.193700,0.169816,0.944342,0.944375
2,0.087700,0.165904,0.946579,0.946584


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ Training complete.
📊 Final Metrics: {'train_runtime': 3064.1355, 'train_samples_per_second': 78.326, 'train_steps_per_second': 1.224, 'total_flos': 1.578694680576e+16, 'train_loss': 0.1610879227002462, 'epoch': 2.0}


In [10]:
trainer.save_model("./results")
tokenizer.save_pretrained("./results")


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

In [11]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import gradio as gr

# ✅ Load tokenizer and model (ensure 'results' folder exists)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("./results")
model.eval()  # ← Required to switch to evaluation mode

# ✅ Prediction function with error handling
def classify_news(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        labels = ["World", "Sports", "Business", "Sci/Tech"]
        return labels[prediction]
    except Exception as e:
        return f"❌ Error: {str(e)}"

# ✅ Launch Gradio interface
interface = gr.Interface(
    fn=classify_news,
    inputs=gr.Textbox(label="Enter News Headline"),
    outputs=gr.Textbox(label="Predicted Topic"),
    title="📰 AG News Classifier (BERT)",
    description="Type a news headline and classify it into: World, Sports, Business, or Sci/Tech."
)

interface.launch()





* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://e0a55ed9c4c443d757.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
from huggingface_hub import notebook_login
notebook_login()


In [14]:
from huggingface_hub import create_repo

# Just pass the repo name directly as a string
create_repo("bert-agnews-classifier", private=False)



RepoUrl('https://huggingface.co/Ali-Zafar/bert-agnews-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Ali-Zafar/bert-agnews-classifier')

In [15]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="./results",        # this is where your model is saved
    repo_id="Ali-Zafar/bert-agnews-classifier",  # replace with your actual username/repo name
    repo_type="model"
)


rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/876M [00:00<?, ?B/s]

...

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ali-Zafar/bert-agnews-classifier/commit/950b0ddadee6544c23465836a3812a45cd6de436', commit_message='Upload folder using huggingface_hub', commit_description='', oid='950b0ddadee6544c23465836a3812a45cd6de436', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ali-Zafar/bert-agnews-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Ali-Zafar/bert-agnews-classifier'), pr_revision=None, pr_num=None)